### Model 1 

In [1]:
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import random as rn
import numpy as np
import pandas as pd

combined_df = pd.read_csv('combined_df.csv') 
combined_df.head(5)


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [2]:
feature_df = pd.read_csv('feature_df.csv') 
feature_df.head(5)

,id,ambg,lex_div,neg,neu,pos,compound,tfid,label
0,411,129.500000,0.958333,0.0000,0.942500,0.057500,0.25530,0.025000,1
1,502,196.000000,0.814815,0.0000,1.000000,0.000000,0.00000,0.040909,1
2,506,281.500000,0.821870,0.0000,0.922500,0.077500,0.43160,0.011544,1
3,530,88.166667,0.953922,0.0000,0.946667,0.053333,0.10195,0.012000,1
4,541,169.500000,0.910401,0.0795,0.876500,0.044000,-0.08895,0.015957,1


In [3]:
cor = feature_df.corr()

print(abs(cor["label"]))

id          0.414768
ambg        0.238026
lex_div     0.225580
neg         0.033130
neu         0.046132
pos         0.025566
compound    0.012478
tfid        0.003490
label       1.000000
Name: label, dtype: float64


In [ ]:
'''
from nltk.tokenize import sent_tokenize

sample_text = (combined_df.iloc[[0]]['text'])[0]
sample_text_sentences = sent_tokenize(sample_text)

print(len(sample_text_sentences))
print(sample_text_sentences[1])
'''

In [4]:
feature_df.columns[1:-1]

Index(['ambg', 'lex_div', 'neg', 'neu', 'pos', 'compound', 'tfid'], dtype='object')

In [5]:
feature_df['label'].describe()

count    23114.000000
mean         0.940685
std          0.236218
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: label, dtype: float64

In [6]:
balanced_df = feature_df.groupby('label').head(1300)
balanced_df['label'].describe()

count    2600.000000
mean        0.500000
std         0.500096
min         0.000000
25%         0.000000
50%         0.500000
75%         1.000000
max         1.000000
Name: label, dtype: float64

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

features = balanced_df.columns[1:-1]

# Separating out the features
x = balanced_df.loc[:, features].values

# Separating out the target
y = np.squeeze(balanced_df.loc[:, ['label']].values)

# Standardizing the features
#x = StandardScaler().fit_transform(x)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
clf = svm.SVC()

print('X_train', len(X_train), 'X_test', len(X_test), 
      'y_train', len(y_train), 'y_test', len(y_test))

X_train 2080 X_test 520 y_train 2080 y_test 520


In [9]:
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)

In [10]:
X_test[0]

array([1.90333333e+02, 9.40642499e-01, 3.90000000e-02, 7.61000000e-01,
       2.00000000e-01, 3.78266667e-01, 1.12903226e-02])

In [11]:
print(confidence)

0.6923076923076923


In [12]:
import pickle
# save the model to disk
filename = 'model1.sav'
pickle.dump(clf, open(filename, 'wb'))

In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

SGDclf = SGDClassifier(max_iter=1000, tol=1e-3, penalty="elasticnet")

In [14]:
SGDclf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None,
              penalty='elasticnet', power_t=0.5, random_state=None,
              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
              warm_start=False)

In [17]:
predicted = SGDclf.predict(X_test)
metrics.confusion_matrix(y_test, predicted)
SGDclf.score(X_test, y_test)

0.6076923076923076

In [18]:
# save the model to disk
filename = 'model1-1.sav'
pickle.dump(SGDclf, open(filename, 'wb'))

In [19]:
SGDclf.predict(X_test[0].reshape(1,-1))

array([1])

In [20]:
from sklearn.linear_model import LogisticRegression

LRmodel = LogisticRegression()
LRmodel.fit(X_train, y_train)

/home/vikram/anaconda3/envs/sinsat/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
LRmodel.score(X_test, y_test)

0.7173076923076923

In [22]:
LRmodel.predict(X_test[0:10])

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1])

In [23]:
# save the model to disk
filename = 'model1-2.sav'
pickle.dump(LRmodel, open(filename, 'wb'))